In [1]:
import numpy as np
import pandas as pd
import deepxde as dde
from deepxde.backend import tf

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.graph_options.rewrite_options.disable_meta_optimizer = True
session = InteractiveSession(config=config)

2023-10-02 17:50:14.980846: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-02 17:50:14.999695: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-02 17:50:14.999711: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-02 17:50:14.999724: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-02 17:50:15.003839: I tensorflow/core/platform/cpu_feature_g

Instructions for updating:
non-resource variables are not supported in the long term


2023-10-02 17:50:15.661844: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-02 17:50:15.674875: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-02 17:50:15.674956: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Enable just-in-time compilation with XLA.



2023-10-02 17:50:15.949018: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-02 17:50:15.949119: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-02 17:50:15.949158: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [10]:
# Carga de datos
xls = pd.ExcelFile("datos_cromatografia.xlsx")
sheet_names = xls.sheet_names
training_data = xls.parse(sheet_names[0])
test_data = xls.parse(sheet_names[1])

# Definición de constantes y parámetros
epsilon = 0.37
u = 5.55  # Convertir a cm/s, si es necesario.
L = 5.55
T_max = max(training_data["min"].values)  # Suponiendo que "min" está en segundos.

# Variables entrenables
K = tf.Variable(1.0, trainable=True)
D_L = tf.Variable(1.0, trainable=True)

C_in = 523.23  # mg/L
t_p = 1.0  # s

def Cp(inputs):
    t = inputs[:, 1:2]
    condition = tf.less_equal(t, [[t_p]])
    return tf.where(condition, C_in * tf.ones_like(t), tf.zeros_like(t))


In [11]:
# Definición de la EDP
def pde(inputs, outputs):
    z, t = inputs[:, 0:1], inputs[:, 1:2]
    C = outputs[:, 0:1]
    
    # Aquí, definimos la relación entre q y C usando la relación de equilibrio
    q = K * C
    
    # Utiliza la ecuación proporcionada para definir la EDP
    dC_dt = dde.grad.jacobian(outputs, inputs, i=0, j=1)
    dC_dz = dde.grad.jacobian(outputs, inputs, i=0, j=0)
    d2C_dz2 = dde.grad.hessian(outputs, inputs, i=0, j=0)
    
    return dC_dt + (1 - epsilon) / epsilon * q + u / epsilon * dC_dz - D_L * d2C_dz2


In [12]:
# Definición del dominio y condiciones de frontera
geom = dde.geometry.Interval(0, L)
timedomain = dde.geometry.TimeDomain(0, T_max)
domain = dde.geometry.GeometryXTime(geom, timedomain)

bc1 = dde.DirichletBC(
    domain, Cp, lambda _, on_boundary: on_boundary, component=0
)

bc2 = dde.NeumannBC(domain, lambda x: 0, lambda _, on_boundary: on_boundary, component=0)

data = dde.data.TimePDE(
    domain,
    pde,
    [bc1, bc2],
    num_domain=1200,
    num_boundary=120,
    num_initial=100,
    train_distribution="pseudo",
)

In [13]:
# Define la red neuronal
layer_size = [2] + [50] * 3 + [1]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.maps.FNN(layer_size, activation, initializer)

# Modelo
model = dde.Model(data, net)

# Entrenamiento
model.compile("adam", lr=0.01)
model.train(epochs=15000)
model.compile("L-BFGS-B")
early_stopping = dde.callbacks.EarlyStopping(min_delta=1e-6, patience=1000)
model.train(callbacks=[early_stopping])

# Valores aprendidos de K y D_L
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    K_value = sess.run(K)
    D_L_value = sess.run(D_L)

print("K aprendido:", K_value)
print("D_L aprendido:", D_L_value)


Compiling model...
Building feed-forward neural network...
'build' took 0.021770 s

'compile' took 0.171504 s



/home/vasco/dev/Python-Projects/HackSciML-g19/venv/lib/python3.10/site-packages/deepxde/nn/tensorflow_compat_v1/fnn.py:116: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(
2023-10-02 17:58:04.229714: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-02 17:58:04.229825: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-02 17:58:04.229878: I tensorflow

Training model...

Step      Train loss                        Test loss                         Test metric
0         [3.75e-01, 8.89e+04, 2.37e-03]    [3.75e-01, 8.89e+04, 2.37e-03]    []  
1000      [7.03e-08, 6.01e+04, 4.98e-11]    [7.03e-08, 6.01e+04, 4.98e-11]    []  
2000      [6.38e-08, 6.01e+04, 1.25e-11]    [6.38e-08, 6.01e+04, 1.25e-11]    []  
3000      [1.13e+03, 2.08e+04, 2.51e+02]    [1.13e+03, 2.08e+04, 2.51e+02]    []  
4000      [1.61e+03, 3.49e+04, 1.41e+02]    [1.61e+03, 3.49e+04, 1.41e+02]    []  
5000      [7.49e+02, 1.17e+04, 1.68e+02]    [7.49e+02, 1.17e+04, 1.68e+02]    []  
6000      [8.18e+02, 1.36e+04, 2.69e+02]    [8.18e+02, 1.36e+04, 2.69e+02]    []  
7000      [8.60e+02, 1.17e+04, 2.76e+02]    [8.60e+02, 1.17e+04, 2.76e+02]    []  
8000      [1.52e+03, 1.03e+04, 2.69e+02]    [1.52e+03, 1.03e+04, 2.69e+02]    []  
9000      [1.11e+03, 1.32e+04, 8.46e+02]    [1.11e+03, 1.32e+04, 8.46e+02]    []  
10000     [7.15e+02, 1.13e+04, 4.00e+02]    [7.15e+02, 1.13e+

2023-10-02 17:58:20.659596: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-02 17:58:20.659702: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-02 17:58:20.659751: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf